In [ ]:
# Import class
import os
import sys

parent_dir = os.path.dirname(os.path.dirname(os.getcwd()))
g_parent_dir = os.path.dirname(parent_dir)
# print(g_parent_dir)

class_dir = os.path.join(g_parent_dir, 'Scripts')
sys.path.insert(0, class_dir)

import datetime as dt
import scraper
import empirical
import pandas as pd

In [ ]:
best_markets = scraper.get_best_player_markets() #dt.date(2019, 1, 31)

## Show markets and order by bookie edge

In [ ]:
bookie_pr_edge_list = []
for market in best_markets:
    bookie_pr_edge_list.append(market.calc_spread())

for edge, market in sorted(zip(bookie_pr_edge_list, best_markets), key=lambda t:t[0]):
    print('Bookie edge {:.2%}'.format(edge))
    print(market, end='\n\n')

## Calculate theos for best markets

In [ ]:
past_n_games = 40
edge_list = []
player_list = []
over_under_list = []
baseline_list = []
stat_list = []
odds_list = []
theo_list = []
bookie_edge_list = []
exchange_list = []
for p_mrkt in best_markets:
        
    try:
        # If unable to calculate theo (e.g. if player's name not correct), move onto next one
        # Calculate theo odds
        theo_odds_over, theo_odds_under = empirical.empirical_odds(p_mrkt.player, p_mrkt.stat, 
                                 p_mrkt.baseline, past_n_games)
    except:
        print('Unable to calculate theo for {} {}'.format(p_mrkt.player,
                                                      p_mrkt.stat))
        continue

    theo_pr_over = 1/theo_odds_over
    theo_pr_under = 1 - theo_pr_over

    # Calculate edge
    edge_over = theo_pr_over * p_mrkt.odds_over - 1
    edge_under = theo_pr_under * p_mrkt.odds_under - 1

    if edge_over > edge_under:
        best_edge = edge_over
        over_under = 'over'
        theo_list.append(round(theo_odds_over,2))
        odds_list.append(p_mrkt.odds_over)
        exchange_list.append(p_mrkt.overs_exchange)
    else:
        best_edge = edge_under
        over_under = 'under'
        theo_list.append(round(theo_odds_under, 2))
        odds_list.append(p_mrkt.odds_under)
        exchange_list.append(p_mrkt.unders_exchange)

    bookie_edge = 1/float(p_mrkt.odds_over) + 1/float(p_mrkt.odds_under) - 1
    
    bookie_edge_list.append(round(bookie_edge, 2))
    edge_list.append(round(best_edge, 2))
    player_list.append(p_mrkt.player)
    over_under_list.append(over_under)
    baseline_list.append(p_mrkt.baseline)
    stat_list.append(p_mrkt.stat)
    
# Create df, sort by edge and print
df_game_mrkts_edge = pd.DataFrame({
    'Edge':edge_list,
    'Spread':bookie_edge_list,
    'Player':player_list,
    'Over_under':over_under_list,
    'Baseline':baseline_list,
    'Stat':stat_list,
    'Odds':odds_list,
    'Theo':theo_list,
    'Exchange':exchange_list
}, columns = ['Edge', 'Spread', 'Player', 'Over_under', 'Baseline', 'Stat', 'Odds', 'Theo', 'Exchange'])

df_game_mrkts_edge = df_game_mrkts_edge.sort_values(by='Spread', ascending=True)
df_game_mrkts_edge

## Record and save in pickle file

In [ ]:
import pickle

if True:
    data_dir = os.path.join(parent_dir, 'Data')
    file_dir = os.path.join(data_dir, 'historic_markets.pickle')

    historic_markets = pickle.load(open(file_dir, 'rb'))

    market_today = [(dt.datetime.now(), best_markets)]

    historic_markets.extend(market_today)

    pickle.dump(historic_markets, open(file_dir, 'wb'))
    
    print('Markets saved at {}'.format(dt.datetime.now()))